In [31]:
import numpy as np
import pandas as pd

In [32]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [33]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,1.0,7.0,3,5,4.0,3.0,3534.0,0,0,2,1.0,2.0,2.25
1,0.0,97.0,2,2,2.0,3.0,950.0,0,0,1,0.0,1.0,0.65
2,0.0,10.0,3,4,3.0,3.0,1305.0,1,0,2,0.0,0.0,1.25
3,0.0,6.0,3,3,3.0,3.0,1311.0,1,0,1,2.0,2.0,1.30
4,1.0,46.0,3,3,3.0,3.0,1570.0,0,0,2,2.0,2.0,1.27


In [34]:
# one hot encode -> sector, balcony, agePossession, furnishing type, luxury category, floor category

In [35]:
x = df.drop(columns = ['price'])
y = df['price']

In [36]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer 
from sklearn.svm import SVR

In [37]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [38]:
# Apply the log1p transformation to the target variable
y_tranformed = np.log1p(y)

In [39]:
# creating a column tranformer for preprocessing
preprocessor = ColumnTransformer(
    transformers =[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop = 'first',handle_unknown='ignore' ), columns_to_encode)
    ],
    remainder = 'passthrough'
)

In [40]:
# creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel = 'rbf'))
])

In [41]:
# k-fold cross-validation
k_fold = KFold(n_splits = 10, shuffle = True, random_state= 42)
score = cross_val_score(pipeline, x, y_tranformed, cv = k_fold, scoring = 'r2')

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [42]:
score.mean()

0.8841704703053285

In [43]:
score.std()

0.016774490136491314

In [44]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x, y_tranformed, test_size= 0.2, random_state= 42)

In [45]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [46]:
y_pred = pipeline.predict(x_test)

In [47]:
y_pred = np.expm1(y_pred)

In [48]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test), y_pred) # the prediction model is making wrong

0.6283827185475316

In [49]:
# try applying another algorithm for more accuracy 